In [ ]:
from Bio import SeqIO

def get_locus_and_genes(genbank_file: str) -> dict[str, list[str]]:
    """
    The get_locus_and_genes function takes a genbank file as input and returns a dictionary of locus names (keys)
    and gene names (values). The function is used to create the locus_gene dictionary that will be used in the
    get_locus_and_genes function. This function is called by get_locus2gene.

    :param genbank_file:str: Specify the file that contains the information about the genes in a genome
    :return: A dictionary with locus names as keys and a list of gene names as values
    """
    locus_gene: dict[str, list[str]] = {}
    handle_gb = open(genbank_file, encoding="utf-8")
    for record in SeqIO.parse(handle_gb, "genbank"):
        locus_gene[record.name] = []
        for feat in record.features:
            if feat.type == "CDS":
                if feat.qualifiers.get("gene", None) == None:
                    locus_gene[record.name].append(feat.qualifiers.get("locus_tag")[0])
                else:
                    locus_gene[record.name].append(feat.qualifiers["gene"][0])
    return locus_gene

get_locus_and_genes("/home/reusebio/tese/insaflu_snakemake/reference/SARS_CoV_2_COVID_19_Wuhan_Hu_1_MN908947.gbk")
get_locus_and_genes("/home/reusebio/tese/insaflu_snakemake/reference/A_H1N1pdm09_A_Michigan_45_2015.gbk")
get_locus_and_genes("/home/reusebio/tese/insaflu_snakemake/reference/monkeypox_MPXV_USA_2022_MA001_ON563414.gbk")

In [8]:
def get_positions_gb(genbank_file):
    """
    The get_positions_gb function takes a genbank file as input and returns a list of lists.
    Each sublist contains the name of the gene, and its start and end positions in that particular
    sequence.
    The get_positions_gb function is called by other functions to retrieve this information.

    :param genbank_file: Open the genbank file and parse it
    :return: A list of lists
    :doc-author: Trelent
    """
    positions = []

    handle_gb = open(genbank_file)
    for record in SeqIO.parse(handle_gb, "genbank"):
        for features in record.features:
            if features.type == "CDS":
                if features.qualifiers.get("gene", None) == None:
                    positions.append([features.qualifiers.get("locus_tag")[0], features.location])
                else:
                    positions.append([features.qualifiers.get("gene")[0], features.location])
    positions_clean = []

    for idx, gene in enumerate(positions):
        positions_clean.append([gene[0], []])
        for part in gene[1].parts:
            positions_clean[idx][1].append([int(part.start), int(part.end)])
    handle_gb.close()
    return positions_clean

get_positions_gb("/home/reusebio/tese/insaflu_snakemake/reference/SARS_CoV_2_COVID_19_Wuhan_Hu_1_MN908947.gbk")
get_positions_gb("/home/reusebio/tese/insaflu_snakemake/reference/A_H1N1pdm09_A_Michigan_45_2015.gbk")
print(get_positions_gb("/home/reusebio/tese/insaflu_snakemake/reference/monkeypox_MPXV_USA_2022_MA001_ON563414.gbk"))

[['MPXV-USA_2022_MA001-001', [[851, 1592]]], ['MPXV-USA_2022_MA001-002', [[1718, 2768]]], ['MPXV-USA_2022_MA001-003', [[2857, 4624]]], ['MPXV-USA_2022_MA001-004', [[4859, 6173]]], ['MPXV-USA_2022_MA001-005', [[6941, 7160]]], ['MPXV-USA_2022_MA001-006', [[7596, 8025]]], ['MPXV-USA_2022_MA001-007', [[9005, 9119]]], ['MPXV-USA_2022_MA001-008', [[9603, 10332]]], ['MPXV-USA_2022_MA001-009', [[10494, 10875]]], ['MPXV-USA_2022_MA001-010', [[10934, 12917]]], ['MPXV-USA_2022_MA001-011', [[13050, 13245]]], ['MPXV-USA_2022_MA001-012', [[13391, 15284]]], ['MPXV-USA_2022_MA001-013', [[15941, 16394]]], ['MPXV-USA_2022_MA001-014', [[16618, 17086]]], ['MPXV-USA_2022_MA001-015', [[17229, 17850]]], ['MPXV-USA_2022_MA001-016', [[17894, 18845]]], ['MPXV-USA_2022_MA001-017', [[18775, 19156]]], ['MPXV-USA_2022_MA001-018', [[19377, 20022]]], ['MPXV-USA_2022_MA001-019', [[20067, 20421]]], ['MPXV-USA_2022_MA001-020', [[20547, 21081]]], ['MPXV-USA_2022_MA001-021', [[21121, 22450]]], ['MPXV-USA_2022_MA001-022', 

In [ ]:
def get_ref_adjusted_positions(alignment, positions, locus, gene):
    """
    The get_ref_adjusted_positions function takes a fasta file of aligned sequences and the positions
    of interest for each gene, and returns the adjusted positions in reference to the first sequence.


    :param alignment: Get the reference sequence
    :param positions: Get the positions of each gene group in the alignment
    :param locus: Identify the gene that is being used to get the positions
    :param gene: Determine which gene is being analyzed
    :return: The positions of the genes in the reference sequence
    :doc-author: Trelent
    """
    references = []
    new_positions = []
    ref = list(SeqIO.parse(alignment, "fasta"))[0]

    index_list = []
    for idx in range(len(ref)):
        if ref[idx] != "-":
            index_list.append(idx)
    for gene_group in positions:
        if gene_group[0] == gene:
            for group in gene_group[1]:
                group[0] = index_list[group[0]]
                if group[1] >= len(
                    index_list
                ):  # if this is == it will throw an error cuz index list é mais pequena que o numero no group[1 ]
                    group[1] = index_list[-1]
                else:
                    # print(len(index_list),group[1])
                    group[1] = index_list[group[1]]
            new_positions.append(gene_group)
    return new_positions

